In [1]:
!nvidia-smi

Mon Nov 27 09:08:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [29]:
import os
HOME = os.getcwd()
print("HOME:", HOME)
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

HOME: /content


In [3]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

  Preparing metadata (setup.py) ... done


In [8]:
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.0 MB/s eta 0:00:00


In [9]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P {HOME}/weights

In [12]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [13]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [14]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

In [15]:
mask_generator = SamAutomaticMaskGenerator(sam)

In [104]:
import os
IMAGE_PATH = '/content/data/GasStation1.jpg'

In [17]:
# helper function that loads an image before adding it to the widget

import base64

def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [105]:
IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget

widget = BBoxWidget()
widget.image = encode_image(IMAGE_PATH)
widget

BBoxWidget(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#b…

In [106]:
widget.bboxes

[{'x': 1187, 'y': 1768, 'width': 438, 'height': 783, 'label': ''}]

In [107]:
import numpy as np

# default_box is going to be used if you will not draw any box on image above
default_box = {'x': 68, 'y': 247, 'width': 555, 'height': 678, 'label': ''}

box = widget.bboxes[0] if widget.bboxes else default_box
box = np.array([
    box['x'],
    box['y'],
    box['x'] + box['width'],
    box['y'] + box['height']
])

In [43]:
predictor = SamPredictor(sam)

In [57]:
# /content/data/Archway1.jpg
image = cv2.imread('/content/data/Archway1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)
print(masks.shape)

image[~masks.reshape(masks.shape[1:])]=255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_fore = '/content/data/Archway1foreground.jpg'
cv2.imwrite(output_file_fore, image)

(1, 3815, 5723)


True

In [58]:
image = cv2.imread('/content/data/Archway1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/Archway1background.jpg'
cv2.imwrite(output_file_back, image)

(3815, 5723, 3)


True

In [52]:
image_archway2 = cv2.imread('/content/data/Archway2.jpg')
image_archway2 = cv2.cvtColor(image_archway2, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image_archway2)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)
print(masks.shape)

image_archway2[~masks.reshape(masks.shape[1:])]=255
image_archway2 = cv2.cvtColor(image_archway2, cv2.COLOR_RGB2BGR)
output_file_fore = '/content/data/Archway2foreground.jpg'
cv2.imwrite(output_file_fore, image_archway2)

(1, 1365, 1817)


True

In [48]:
image = cv2.imread('/content/data/Archway2.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/Archway2background.jpg'
cv2.imwrite(output_file_back, image)

(1365, 1817, 3)


True

In [63]:
image = cv2.imread('/content/data/Airplane1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/Airplane1background.jpg'
cv2.imwrite(output_file_back, image)

(1071, 1600, 3)


True

In [65]:
image = cv2.imread('/content/data/Airplane1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)

image[~masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_fore = '/content/data/Airplane1foreground.jpg'
cv2.imwrite(output_file_fore, image)

(1071, 1600, 3)


True

In [83]:
image = cv2.imread('/content/data/Airplane2.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)

image[~masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_fore = '/content/data/Airplane2foreground.jpg'
cv2.imwrite(output_file_fore, image)

(1491, 2572, 3)


True

In [72]:
image = cv2.imread('/content/data/Airplane2.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/Airplane2background.jpg'
cv2.imwrite(output_file_back, image)

(1491, 2572, 3)


True

In [80]:
image = cv2.imread('/content/data/Bag1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)
image = image.astype(np.uint16)
print(masks.shape)
image[masks[0],1] *= 2

image = np.minimum(image, 255).astype(np.uint8)
# masks = masks.reshape(masks.shape[1:])
# print(masks.shape)

# masks = masks.astype(np.uint8) * 255
# color_mask = cv2.cvtColor(masks, cv2.COLOR_GRAY2BGR)
# color_mask[:, :, 1] = 255  # Set the green channel to 255

# print(color_mask.shape, image.shape)
# result = cv2.addWeighted(image, 1, color_mask, 0.1, 0)
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
cv2.imwrite('/content/data/maskedBag1.jpg', image)



(1, 2592, 1944)


True

In [81]:
#bag1-fore
image = cv2.imread('/content/data/Bag1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)

image[~masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_fore = '/content/data/Bag1foreground.jpg'
cv2.imwrite(output_file_fore, image)

(2592, 1944, 3)


True

In [82]:
#bag1-back
image = cv2.imread('/content/data/Bag1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/Bag1background.jpg'
cv2.imwrite(output_file_back, image)

(2592, 1944, 3)


True

In [90]:
#Bridge1

image = cv2.imread('/content/data/Bridge1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)

image[~masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_fore = '/content/data/Bridge1foreground.jpg'
cv2.imwrite(output_file_fore, image)


(3380, 5328, 3)


True

In [91]:

image = cv2.imread('/content/data/Bridge1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/Bridge1background.jpg'
cv2.imwrite(output_file_back, image)

(3380, 5328, 3)


True

In [96]:
#Clothes1

image = cv2.imread('/content/data/Clothes1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)

image[~masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_fore = '/content/data/Clothes1foreground.jpg'
cv2.imwrite(output_file_fore, image)


(533, 800, 3)


True

In [97]:

image = cv2.imread('/content/data/Clothes1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/Clothes1background.jpg'
cv2.imwrite(output_file_back, image)

(533, 800, 3)


True

In [102]:
#Fence1

image = cv2.imread('/content/data/Fence1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)

image[~masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_fore = '/content/data/Fence1foreground.jpg'
cv2.imwrite(output_file_fore, image)


(1098, 1600, 3)


True

In [103]:

image = cv2.imread('/content/data/Fence1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/Fence1background.jpg'
cv2.imwrite(output_file_back, image)

(1098, 1600, 3)


True

In [110]:
#GasStation1

image = cv2.imread('/content/data/GasStation1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)

print(masks.shape, masks.max())
image[~masks.reshape(masks.shape[1:])] =255
output_file_fore = '/content/data/GasStation1foreground.jpg'
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
cv2.imwrite(output_file_fore, image)


(1, 2920, 4032) True


True

In [112]:

image = cv2.imread('/content/data/GasStation1.jpg')
print(image.shape)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,
)

image[masks.reshape(masks.shape[1:])] =255
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
output_file_back = '/content/data/GasStation1background.jpg'
cv2.imwrite(output_file_back, image)

(2920, 4032, 3)


True

In [113]:
image = cv2.imread('/content/data/GasStation1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_box = np.array([box])
input_point, input_label = None, None
predictor.set_image(image)
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    box = input_box,
    multimask_output=False,

)
image = image.astype(np.uint16)
print(masks.shape)
image[masks[0],1] *= 2

image = np.minimum(image, 255).astype(np.uint8)
# masks = masks.reshape(masks.shape[1:])
# print(masks.shape)

# masks = masks.astype(np.uint8) * 255
# color_mask = cv2.cvtColor(masks, cv2.COLOR_GRAY2BGR)
# color_mask[:, :, 1] = 255  # Set the green channel to 255

# print(color_mask.shape, image.shape)
# result = cv2.addWeighted(image, 1, color_mask, 0.1, 0)
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
cv2.imwrite('/content/data/maskedGasStation1.jpg', image)



(1, 2920, 4032)


True